In [6]:
import logging
import json
import torch
from types import SimpleNamespace
from vilbert.vilbert import VILBertActionGrounding, BertConfig
from pytorch_transformers.tokenization_bert import BertTokenizer
from pytorch_transformers.optimization import AdamW, WarmupLinearSchedule
import torch.distributed as dist
from VLN_config import config as args

In [9]:
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)
logger = logging.getLogger(__name__)
print(args)

namespace(adam_epsilon=1e-08, baseline=False, bert_model='bert-base-uncased', best_features=10, clean_train_sets=True, config_file='config/bert_base_6layer_6conect.json', distributed=False, do_lower_case=True, dynamic_attention=False, from_pretrained='save/multitask_model/multi_task_model.bin', gradient_accumulation_steps=1, img_weight=1, in_memory=False, learning_rate=0.0001, local_rank=-1, max_temporal_memory_buffer=5, mean_layer=True, num_train_epochs=10.0, num_workers=0, objective=1, predict_feature=False, save_name='', seed=42, split='mteval', start_epoch=0, task_specific_tokens=True, tasks='1', threshold_similarity=0.65, track_temporal_features=True, train_batch_size=1, visual_target=0, warmup_proportion=0.1, without_coattention=False)


In [ ]:
# Input:
# - instruction.
# - image.

# train_dataset =
# val_dataset =


In [16]:
config = BertConfig.from_json_file(args.config_file)
bert_weight_name = json.load(
    open("config/" + args.bert_model + "_weight_name.json", "r")
)

tokenizer = BertTokenizer.from_pretrained(
    args.bert_model, do_lower_case=args.do_lower_case
)

config.track_temporal_features = args.track_temporal_features
config.mean_layer = args.mean_layer
config.max_temporal_memory_buffer = args.max_temporal_memory_buffer


model = VILBertActionGrounding.from_pretrained(
    args.from_pretrained, config=config, default_gpu=True
)

07/14/2020 17:49:20 - INFO - pytorch_transformers.tokenization_utils -   loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/drissh/.cache/torch/pytorch_transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
07/14/2020 17:49:20 - INFO - vilbert.utils -   loading weights file save/multitask_model/multi_task_model.bin
07/14/2020 17:49:31 - INFO - vilbert.utils -   Weights of VILBertActionGrounding not initialized from pretrained model: ['positional_enc.weight', 'positional_enc.bias', 'img_emb_mean.weight', 'img_emb_mean.bias', 'action_cls.predictions.bias', 'action_cls.predictions.transform.dense.weight', 'action_cls.predictions.transform.dense.bias', 'action_cls.predictions.transform.LayerNorm.weight', 'action_cls.predictions.transform.LayerNorm.bias', 'action_cls.predictions.decoder.weight', 'action_cls.bi_seq_relationship.weight', 'action

In [17]:
model

VILBertActionGrounding(
  (positional_enc): Linear(in_features=7, out_features=2048, bias=True)
  (img_emb_mean): Linear(in_features=10240, out_features=2048, bias=True)
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (v_embeddings): BertImageEmbeddings(
      (image_embeddings): Linear(in_features=2048, out_features=1024, bias=True)
      (image_location_embeddings): Linear(in_features=5, out_features=1024, bias=True)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True

In [ ]:
for key, value in dict(model.named_parameters()).items():
    if not value.requires_grad:
        print("This parameter does have grad", key)

In [30]:
no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
config.visual_target = args.visual_target

optimizer_grouped_parameters = []
for key, value in dict(model.named_parameters()).items():
    if value.requires_grad:
        if "cls" in key:
            lr = args.learning_rate
        else:
            lr = args.learning_rate * 0.1
        if any(nd in key for nd in no_decay): # No decay
            optimizer_grouped_parameters += [
                {"params": [value],
                 "lr": lr,
                 "weight_decay": 0.0}
            ]
        
        elif not any(nd in key for nd in no_decay):
            optimizer_grouped_parameters += [
                {"params": [value],
                 "lr": lr,
                 "weight_decay": 0.01}
            ]

optimizer = AdamW(
    optimizer_grouped_parameters,
    lr=args.learning_rate,
    eps=args.adam_epsilon,
    betas=(0.9, 0.98),
)

num_dataset_points = 19

num_train_optimization_steps = int(
    num_dataset_points
    / args.train_batch_size
    / args.gradient_accumulation_steps
) * (args.num_train_epochs - args.start_epoch)

scheduler = WarmupLinearSchedule(
    optimizer,
    warmup_steps=args.warmup_proportion * num_train_optimization_steps,
    t_total=num_train_optimization_steps,
)

In [18]:
# model.cuda()
# for state in optimizer.state.values():
#     for k, v in state.items():
#         if torch.is_tensor(v):
#             state[k] = v.cuda()
logger.info("***** Running training *****")
logger.info("Num examples = %d", num_dataset_points)
logger.info("Batch size = %d", args.train_batch_size)
logger.info("Num steps = %d", num_train_optimization_steps)

07/14/2020 17:51:37 - INFO - __main__ -   ***** Running training *****


NameError: name 'num_dataset_points' is not defined

In [ ]:
model

In [ ]:
startIterID = 0
global_step = 0

for epoch in range(int(args.start_epoch), int(args.num_train_epochs)):
    model.train()
    for step, batch in enumerate(train_dataset):
        iterId = startIterID + step + (epochId * len(train_dataset))
    